# Add Image Metrics to an Existing Table

In this example, we will first write a simple table containing a single column of image paths.
We will then write a second table, containing the image paths from the first table, and extended with new columns containing image metrics.

In [ ]:
from pathlib import Path
import tlc
from PIL import Image, ImageStat
import numpy as np

## Write the initial table

In [ ]:
data_path = Path("../data/coco128/images").absolute().as_posix()
dataset_name = "coco128"
project_name = "add-image-metrics"

table = tlc.Table.from_image_folder(
    data_path,
    table_name="initial",
    dataset_name=dataset_name,
    project_name=project_name,
    include_label_column=False,
    add_weight_column=False,
    description="Initial table with images from COCO128 dataset",
)

## Extend the table with image metrics

In [ ]:
def compute_image_metrics(image_path: str):    
    """Return a dict of image metrics for the given image path."""
    image = Image.open(image_path)
    width, height = image.size
    pixels = np.array(image)

    # Convert to grayscale for some metrics
    grayscale_image = image.convert("L")
    stat = ImageStat.Stat(grayscale_image)

    # Compute brightness (average grayscale value)
    brightness = stat.mean[0]

    # Compute contrast (standard deviation of grayscale values)
    contrast = stat.stddev[0]

    # Compute average RGB values
    try:
        avg_r = np.mean(pixels[:, :, 0])
        avg_g = np.mean(pixels[:, :, 1])
        avg_b = np.mean(pixels[:, :, 2])
    except IndexError: # Image is grayscale
        avg_r = avg_g = avg_b = 0

    return {
        "width": width,
        "height": height,
        "brightness": brightness,
        "contrast": contrast,
        "average_red": avg_r,
        "average_green": avg_g,
        "average_blue": avg_b,
    }

In [ ]:
# Create a TableWriter for the extended table
# We ensure the schema of the metrics-columns are not writeable and hidden in the sample-view
extended_table_writer = tlc.TableWriter(
    table_name="added-image-metrics",
    dataset_name=dataset_name,
    project_name=project_name,
    description="COCO128 dataset with added image metrics",
    column_schemas={
        "image": tlc.ImagePath,
        "width": tlc.Schema(value=tlc.Int32Value(), writable=False, sample_type="hidden"),
        "height": tlc.Schema(value=tlc.Int32Value(), writable=False, sample_type="hidden"),
        "brightness": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
        "contrast": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
        "average_red": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
        "average_green": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
        "average_blue": tlc.Schema(value=tlc.Float32Value(), writable=False, sample_type="hidden"),
    },
)

# Iterate through the input table, compute metrics for each image, and add the metrics to the extended table
for row in table.table_rows:
    image_path = row["image"]
    metrics = compute_image_metrics(image_path)
    new_row = {**row, **metrics}
    extended_table_writer.add_row(new_row)

extended_table = extended_table_writer.finalize()

print(extended_table[0].keys())            # Notice only the "image" column is present in the "sample-view" of the table
print(extended_table.table_rows[0].keys()) # Notice all the columns are present in the "row-view" of the table